In [2]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors



In [3]:
file_path = "/gluster/data/dune/niclane/nlane_prod_strange_resample_fhc_run2_fhc_reco2_reco2_trainingimage_background_lambdamuon_ana.root"  
#file_path = "/gluster/data/dune/niclane/test.root"

root_file = uproot.open(file_path)
print(root_file.keys())

['imageanalyser;1', 'imageanalyser/SampleTree;1', 'imageanalyser/ImageTree;1', 'FRH;1', 'FRV;1', 'rICKR;1', 'rICKI;1', 'PreC;1', 'PostC;1', 'PostO;1', 'PreD;1', 'PostDO;1', 'ER;1']


In [4]:
tree_name = "imageanalyser/ImageTree" 
tree = root_file[tree_name]

print(tree.keys())

['run', 'subrun', 'event', 'event_type', 'planes', 'width', 'height', 'input_data', 'truth_data']


In [5]:
input_data = tree["input_data"].array(library="np")
truth_data = tree["truth_data"].array(library="np")

print("Shape:", input_data[1])
print("Data Type:", type(input_data))

Shape: [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], ...]
Data Type: <class 'numpy.ndarray'>


In [6]:
print(f"Number of events: {len(input_data)}")  
print(f"Planes per event: {len(input_data[0])}")  
print(f"Pixels per plane: {len(input_data[0][0])}")

Number of events: 706
Planes per event: 3
Pixels per plane: 262144


In [7]:
event_index = 2
event = input_data[event_index]  
truth_event = truth_data[event_index] 

run_numbers = tree["run"].array(library="np")
subrun_numbers = tree["subrun"].array(library="np")
event_numbers = tree["event"].array(library="np")

event_type = tree["event_type"].array(library="np")

print(event_type[event_index])

print(type(input_data))
plane = event[0]
print(type(plane))

num_entries = tree.num_entries
print(f"Number of entries in {tree_name}: {num_entries}")

1
<class 'numpy.ndarray'>
<class 'uproot.containers.STLVector'>
Number of entries in imageanalyser/ImageTree: 706


In [8]:
W, H = 512, 512  

plane_images = [np.array(list(plane), dtype=np.float32).reshape(H, W) for plane in event]
truth_images = [np.array(list(plane), dtype=np.float32).reshape(H, W) for plane in truth_event]

plane_labels = ["U", "V", "W"]

r, sr, evnum = run_numbers[event_index], subrun_numbers[event_index], event_numbers[event_index]

for i, (input_img, truth_img) in enumerate(zip(plane_images, truth_images)):
    fig, ax = plt.subplots(figsize=(12, 12), dpi=600)

    ax.imshow(input_img,
              origin="lower",
              cmap="jet",
              norm=colors.PowerNorm(gamma=0.35, vmin=input_img.min(), vmax=input_img.max()))

    overlay = False
    if overlay:
        ax.imshow(truth_img, origin="lower", cmap="cool", alpha=0.4)

    ax.set_xticks([0, W - 1])
    ax.set_yticks([0, H - 1])
    ax.tick_params(axis="both", direction="out", length=6, width=1.5, labelsize=18)
    ax.set_xlim(0, W - 1)
    ax.set_ylim(0, H - 1)
    ax.set_xlabel("Local Wire Coord", fontsize=20)
    ax.set_ylabel("Local Drift Time", fontsize=20)
    ax.set_title(f"Plane {plane_labels[i]} (Run {r}, Subrun {sr}, Event {evnum})", fontsize=22)

    plt.tight_layout()
    plt.savefig(f"event_{r}_{sr}_{evnum}_plane_{plane_labels[i]}.png")
    plt.close(fig)

In [10]:
event_indices = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

run_numbers = tree["run"].array(library="np")
subrun_numbers = tree["subrun"].array(library="np")
event_numbers = tree["event"].array(library="np")
event_type = tree["event_type"].array(library="np")

W, H = 512, 512  
plane_labels = ["U", "V", "W"]

for event_index in event_indices:
    event = input_data[event_index]  
    truth_event = truth_data[event_index]  

    print(f"Processing Event Index: {event_index}")
    print(f"Event Type: {event_type[event_index]}")

    r, sr, evnum = run_numbers[event_index], subrun_numbers[event_index], event_numbers[event_index]

    plane_images = [np.array(list(plane), dtype=np.float32).reshape(H, W) for plane in event]
    truth_images = [np.array(list(plane), dtype=np.float32).reshape(H, W) for plane in truth_event]

    for i, (input_img, truth_img) in enumerate(zip(plane_images, truth_images)):
        fig, ax = plt.subplots(figsize=(12, 12), dpi=600)

        ax.imshow(input_img,
                  origin="lower",
                  cmap="jet",
                  norm=colors.PowerNorm(gamma=0.35, vmin=input_img.min(), vmax=input_img.max()))

        overlay = True
        if overlay:
            ax.imshow(truth_img, origin="lower", cmap="cool", alpha=0.4)

        ax.set_xticks([0, W - 1])
        ax.set_yticks([0, H - 1])
        ax.tick_params(axis="both", direction="out", length=6, width=1.5, labelsize=18)
        ax.set_xlim(0, W - 1)
        ax.set_ylim(0, H - 1)
        ax.set_xlabel("Local Wire Coord", fontsize=20)
        ax.set_ylabel("Local Drift Time", fontsize=20)
        ax.set_title(f"Plane {plane_labels[i]} (Run {r}, Subrun {sr}, Event {evnum})", fontsize=22)

        plt.tight_layout()
        plt.savefig(f"event_{r}_{sr}_{evnum}_plane_{plane_labels[i]}.png")
        plt.close(fig)

Processing Event Index: 3
Event Type: 1
Processing Event Index: 4
Event Type: 1
Processing Event Index: 5
Event Type: 1
Processing Event Index: 6
Event Type: 1
Processing Event Index: 7
Event Type: 1
Processing Event Index: 8
Event Type: 1
Processing Event Index: 9
Event Type: 1
Processing Event Index: 10
Event Type: 1
Processing Event Index: 11
Event Type: 1
Processing Event Index: 12
Event Type: 1
Processing Event Index: 13
Event Type: 1
Processing Event Index: 14
Event Type: 1
Processing Event Index: 15
Event Type: 1
Processing Event Index: 16
Event Type: 1
Processing Event Index: 17
Event Type: 1
Processing Event Index: 18
Event Type: 1
Processing Event Index: 19
Event Type: 1
Processing Event Index: 20
Event Type: 1
